# Smith - Waterman Algorithm


Sequence alignment is a powerful tool in bioinformatics used to uncover functional, structural, and evolutionary relationships. It is the process of comparing two or more biological sequences, typically DNA, RNA, or protein sequences to identify regions of similarity. With biological sequences, it is often necessary to align two sequences that are of different lengths, or that have regions that have been inserted or deleted over time. Thus, the notion of gaps needs to be introduced. In this case, a gap is denoted in the alignment by a '-' character. An alignment can produce one of the following: a match between two characters, a mismatch between two characters (also called a substitution or mutation), a gap in the first sequence (which can be thought of as the deletion of a character in the first sequence), or a gap in the second sequence (which can be thought of as the insertion of a character in the first sequence). By aligning sequences globally or locally, researchers can gain insights into how genes, proteins, or entire genomes are related across species, how they evolve, and how their structures and functions are conserved.

Local alignment finds the region of highest similarity between two sequences, often focusing on shorter subsequences. This method is useful when sequences contain conserved regions but have diverged significantly outside those regions. In 1981, Temple Smith and Mike Waterman proposed a modification
to the Needleman-Wunsch algorithm in order to obtain a local
sequence alignment resulting in the highest scoring local match
between two sequences. The Smith-Waterman algorithm does not penalize gaps or mismatches outside of the optimal local alignment.

Here's how we can implement it step by step:

1. **Initialization**: At first, we have to define the sequences (`seq1` and `seq2`), and the scoring parameters (`match_score`, `mismatch_score` and `gap_penalty`). Then a scoring matrix needs to be created with dimensions (`len(seq1) + 1) x (len(seq2) + 1`). All the cells of the scoring matrix are filled in with 0s.

2. **Matrix Filling**: The scoring matrix is filled using dynamic programming, ensuring that no cell value goes below 0. For each cell (i, j), the score is calculated as follows:

  $S_{(i, j)}$ = MAXIMUM $[$
  $S_{(i - 1, j - 1)}$ + `match or mismatch score`,
  $S_{(i, j - 1)}$ + `gap_penalty`,
  $S_{(i - 1, j)}$ + `gap_penalty`, 0 $]$

3. **Traceback**: Once the matrix is filled, the optimal alignment is obtained by tracing back from the cell with the highest score. The traceback stops when we hit a cell with a score of 0. During this process, we'll construct two aligned sequences by following the optimal path, which is based on the scoring rules. The trace gives us the alignment, which is the optimal match of both sequences. For traceback, we move diagonally when it's a match or mismatch, up when there's a gap in seq2, and left when there's a gap in seq1.


In [4]:
import numpy as np
import pandas as pd

In [5]:
def smith_waterman(seq1, seq2):
    # Scoring parameters
    match_score, mismatch_score, gap_penalty = 5, -3, -4

    # Initialize the scoring matrix
    len_m, len_n = len(seq1), len(seq2)
    scoring_matrix = [[0 for j in range(len_m + 1)] for i in range(len_n + 1)]

    # Keep track of the maximum score and its position for traceback
    max_score = 0
    max_pos = (0, 0)

    # Fill in the scoring matrix
    for i in range(1, len_n + 1):
        for j in range(1, len_m + 1):
            # print(i, j)
            if seq2[i - 1] == seq1[j - 1]:
                match = scoring_matrix[i - 1][j - 1] + match_score
            else:
                match = scoring_matrix[i - 1][j - 1] + mismatch_score
            delete = scoring_matrix[i - 1][j] + gap_penalty
            insert = scoring_matrix[i][j - 1] + gap_penalty
            scoring_matrix[i][j] = max(match, delete, insert, 0)
            if scoring_matrix[i][j] > max_score:
                max_score = scoring_matrix[i][j]
                max_pos = (i, j)

    # Traceback to build the alignment
    aligned_seq1 = ""
    aligned_seq2 = ""
    i, j = max_pos

    while i > 0 and j > 0 and scoring_matrix[i][j] != 0:
        current_score = scoring_matrix[i][j]
        diagonal_score = scoring_matrix[i - 1][j - 1]
        up_score = scoring_matrix[i - 1][j]
        left_score = scoring_matrix[i][j - 1]

        # Check if diagonal (match or mismatch)
        if seq2[i - 1] == seq1[j - 1]:
            score = match_score
        else:
            score = mismatch_score

        if current_score == diagonal_score + score:
            aligned_seq2 = seq2[i - 1] + aligned_seq2
            aligned_seq1 = seq1[j - 1] + aligned_seq1
            i -= 1
            j -= 1
        elif current_score == up_score + gap_penalty:
            aligned_seq2 = seq2[i - 1] + aligned_seq2
            aligned_seq1 = "-" + aligned_seq1
            i -= 1
        else:  # current_score == left_score + gap_penalty
            aligned_seq2 = "-" + aligned_seq2
            aligned_seq1 = seq1[j - 1] + aligned_seq1
            j -= 1

    return aligned_seq1, aligned_seq2, scoring_matrix, max_score


## Testing the Algorithm

In [6]:
seq1 = "GAATTCAGTTA"
seq2 = "GGATCGA"
aligned_seq1, aligned_seq2, scoring_matrix, alignment_score = smith_waterman(seq1, seq2)
print(f"Sequence 1: {aligned_seq1}")
print(f"Sequence 2: {aligned_seq2}")
# for row in scoring_matrix:
#      print (row)
# print(*scoring_matrix, sep = '\n')
print(np.matrix(scoring_matrix))
print(f"Optimal Alignment Score: {alignment_score}")

Sequence 1: GAATTCAG
Sequence 2: GGA-TC-G
[[ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  1  0  0  0  0  0  5  1  0  0]
 [ 0  5  2  0  0  0  0  0  5  2  0  0]
 [ 0  1 10  7  3  0  0  5  1  2  0  5]
 [ 0  0  6  7 12  8  4  1  2  6  7  3]
 [ 0  0  2  3  8  9 13  9  5  2  3  4]
 [ 0  5  1  0  4  5  9 10 14 10  6  2]
 [ 0  1 10  6  2  1  5 14 10 11  7 11]]
Optimal Alignment Score: 14
